In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import accuracy_score,f1_score,r2_score,mean_squared_error, recall_score, precision_score
import matplotlib.pyplot as plt
from joblib import dump

In [ ]:
from sklearn.metrics import confusion_matrix

def multiclass_csi(y_true, y_pred):
    cm = confusion_matrix(y_true, y_pred)
    num_classes = cm.shape[0]
    H = np.zeros(num_classes)
    FM = np.zeros(num_classes)

    for i in range(num_classes):
        H[i] = cm[i, i]
        FM[i] = cm[i, :].sum() - H[i]

    H = H.sum() - cm[-1, -1]
    FM = FM.sum()
    csi = H / (H+FM)
    return csi

from sklearn.metrics import make_scorer

csi_scorer = make_scorer(multiclass_csi, greater_is_better=True)

In [ ]:
test = pd.read_csv('/content/drive/MyDrive/기상청빅데이터/DATA/fog_test.csv')

In [ ]:
import joblib
import torch
from sklearn.preprocessing import StandardScaler

# PCA 모델 로드
pca_model_A = joblib.load('/content/drive/MyDrive/기상청빅데이터/과제 2/Model/loadmodel/pca_model_A.joblib')
pca_model_B = joblib.load('/content/drive/MyDrive/기상청빅데이터/과제 2/Model/loadmodel/pca_model_B.joblib')
pca_model_C = joblib.load('/content/drive/MyDrive/기상청빅데이터/과제 2/Model/loadmodel/pca_model_C.joblib')
pca_model_D = joblib.load('/content/drive/MyDrive/기상청빅데이터/과제 2/Model/loadmodel/pca_model_D.joblib')
pca_model_E = joblib.load('/content/drive/MyDrive/기상청빅데이터/과제 2/Model/loadmodel/pca_model_E.joblib')

# Standard Scaler 로드
sd_scaler = joblib.load('/content/drive/MyDrive/기상청빅데이터/과제 2/Model/loadmodel/sd_scaler.bin')

# XGBoost 모델 로드
xgb_model_A = joblib.load('/content/drive/MyDrive/기상청빅데이터/과제 2/Model/loadmodel/xgb_5/xgb_a(6).pt')
xgb_model_B = joblib.load('/content/drive/MyDrive/기상청빅데이터/과제 2/Model/loadmodel/xgb_5/xgb_b(3.3).pt')
xgb_model_C = joblib.load('/content/drive/MyDrive/기상청빅데이터/과제 2/Model/loadmodel/xgb_2/xgb_c2.pt')
xgb_model_D = joblib.load('/content/drive/MyDrive/기상청빅데이터/과제 2/Model/loadmodel/xgb_3/xgb_d3.pt')
xgb_model_E = joblib.load('/content/drive/MyDrive/기상청빅데이터/과제 2/Model/loadmodel/xgb_5/xgb_e(6).pt')

# 각 모델의 상태를 확인하기 위한 출력
print(pca_model_A)
print(pca_model_B)
print(pca_model_C)
print(pca_model_D)
print(pca_model_E)

print(sd_scaler)

print(xgb_model_A)
print(xgb_model_B)
print(xgb_model_C)
print(xgb_model_D)
print(xgb_model_E)

PCA(n_components=5)
PCA(n_components=5)
PCA(n_components=5)
PCA(n_components=5)
PCA(n_components=5)
StandardScaler()
XGBClassifier(alpha=0, base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.9, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=0.05, grow_policy=None, importance_type=None,
              interaction_constraints=None, lambda=0.1, learning_rate=0.05,
              max_bin=None, max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=17, max_leaves=None,
              min_child_weight=3, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None, ...)
XGBClassifier(alpha=0, base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.9, 

# test 전처리(돌릴때 스킵하고 돌려도 됨)

In [ ]:
test

,Unnamed: 0,fog_test.year,fog_test.month,fog_test.day,fog_test.time,fog_test.minute,fog_test.stn_id,fog_test.ws10_deg,fog_test.ws10_ms,fog_test.ta,fog_test.re,fog_test.hm,fog_test.sun10,fog_test.ts,fog_test.class
0,1,L,1,1,0,0,AI,329.5,0.6,-2.5,0.0,87.5,0.0,-1.6,NaN
1,2,L,1,1,0,10,AI,321.8,1.2,-2.5,0.0,88.2,0.0,-1.6,NaN
2,3,L,1,1,0,20,AI,0.4,0.4,-2.5,0.0,88.6,0.0,-1.7,NaN
3,4,L,1,1,0,30,AI,323.6,0.7,-2.6,0.0,88.7,0.0,-1.6,NaN
4,5,L,1,1,0,40,AI,208.4,0.2,-2.6,0.0,89.0,0.0,-1.6,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
262795,262796,L,12,31,23,10,ED,338.8,6.1,4.5,0.0,71.3,0.0,1.9,NaN
262796,262797,L,12,31,23,20,ED,335.2,5.8,4.6,0.0,71.4,0.0,1.9,NaN
262797,262798,L,12,31,23,30,ED,337.1,5.3,4.5,0.0,72.1,0.0,1.8,NaN
262798,262799,L,12,31,23,40,ED,322.0,3.7,4.3,0.0,72.5,0.0,1.4,NaN


In [ ]:
# 결측치 np.nan으로 변경

test = test.replace(-99.9, np.nan)
test = test.replace(-99, np.nan)
test.isnull().sum()

Unnamed: 0                0
fog_test.year             0
fog_test.month            0
fog_test.day              0
fog_test.time             0
fog_test.minute           0
fog_test.stn_id           0
fog_test.ws10_deg       224
fog_test.ws10_ms        224
fog_test.ta              91
fog_test.re            4503
fog_test.hm              57
fog_test.sun10          115
fog_test.ts            7643
fog_test.class       262800
dtype: int64

In [ ]:
# dataframe의 칼럼명을 담은 인덱스 리스트생성

test_var = test.columns
test_var = test_var[1:]
print(test_var)

# 이산변수만 담은 리스트
test_var_dis = [test_var[i] for i in [0, 1, 2, 3, 4, 5, 9, 13]]


# 연속변수만 리은 리스트
test_var_con = [item for item in test_var if item not in test_var_dis]
print(test_var_con)

test_var_dis.pop() # class 제외!
print(test_var_dis)

Index(['fog_test.year', 'fog_test.month', 'fog_test.day', 'fog_test.time',
       'fog_test.minute', 'fog_test.stn_id', 'fog_test.ws10_deg',
       'fog_test.ws10_ms', 'fog_test.ta', 'fog_test.re', 'fog_test.hm',
       'fog_test.sun10', 'fog_test.ts', 'fog_test.class'],
      dtype='object')
['fog_test.ws10_deg', 'fog_test.ws10_ms', 'fog_test.ta', 'fog_test.hm', 'fog_test.sun10', 'fog_test.ts']
['fog_test.year', 'fog_test.month', 'fog_test.day', 'fog_test.time', 'fog_test.minute', 'fog_test.stn_id', 'fog_test.re']


In [ ]:
# 기상변수는 무작위 결측 (관측기기간 상관관계 없음)

# 연속변수
## 이전 행 값 사용
for var in test_var_con :
  test[str(var)] = test[str(var)].interpolate(method='pad')

# 이산변수
## 이전 행 값 사용
for var in test_var_dis :
  test[str(var)] = test[str(var)].interpolate(method='pad')


In [ ]:
test.isnull().sum() # class 제외 결측치 처리 완.

Unnamed: 0                0
fog_test.year             0
fog_test.month            0
fog_test.day              0
fog_test.time             0
fog_test.minute           0
fog_test.stn_id           0
fog_test.ws10_deg         0
fog_test.ws10_ms          0
fog_test.ta               0
fog_test.re               0
fog_test.hm               0
fog_test.sun10            0
fog_test.ts               0
fog_test.class       262800
dtype: int64

In [ ]:
# datetime 생성을 위해 new_year 칼럼 생성

condition = [ (test['fog_test.year'] == 'L') ]
value = [ 2023 ]

test['new_year'] = np.select(condition, value)
test['new_year'].value_counts()

new_year
2023    262800
Name: count, dtype: int64

In [ ]:
# 각 칼럼을 합쳐서 datetime으로 변환

test['datetime'] = test['new_year'].map(str) + "-" + test['fog_test.month'].astype(str).str.zfill(2) + "-" + test['fog_test.day'].astype(str).str.zfill(2) + " " + test['fog_test.time'].astype(str).str.zfill(2) + ":" + test['fog_test.minute'].map(str)
test['datetime'] = pd.to_datetime(test.datetime)
test['datetime']

0        2023-01-01 00:00:00
1        2023-01-01 00:10:00
2        2023-01-01 00:20:00
3        2023-01-01 00:30:00
4        2023-01-01 00:40:00
                 ...        
262795   2023-12-31 23:10:00
262796   2023-12-31 23:20:00
262797   2023-12-31 23:30:00
262798   2023-12-31 23:40:00
262799   2023-12-31 23:50:00
Name: datetime, Length: 262800, dtype: datetime64[ns]

In [ ]:
# 일별 강수강도, 당일 최고 최저기온, 일사량 계산시 사용

test['date'] = test['datetime'].dt.date

In [ ]:
test.isnull().sum() # class 제외 결측치 처리 완.

Unnamed: 0                0
fog_test.year             0
fog_test.month            0
fog_test.day              0
fog_test.time             0
fog_test.minute           0
fog_test.stn_id           0
fog_test.ws10_deg         0
fog_test.ws10_ms          0
fog_test.ta               0
fog_test.re               0
fog_test.hm               0
fog_test.sun10            0
fog_test.ts               0
fog_test.class       262800
new_year                  0
datetime                  0
date                      0
dtype: int64

In [ ]:
# 필요없는 거 삭제

test = test.drop(columns = ['Unnamed: 0', 'fog_test.year', 'fog_test.day', 'new_year'])

In [ ]:
# 풍향 Fourier feature

import math

test['fog_test.ws10_deg_cos'] = test['fog_test.ws10_deg'].apply(lambda x : math.cos(x)) # x-axis
test['fog_test.ws10_deg_sin'] = test['fog_test.ws10_deg'].apply(lambda x : math.sin(x)) # y-axis

In [ ]:
# 시간 + 분을 시간 한개로 합치기

test['fog_test.minute'] = test['fog_test.minute'] / 60
test['time'] = test['fog_test.time'] + test['fog_test.minute']

In [ ]:
# 시간 Fourier feature

test['time_sin'] = test['time'].apply(lambda x: np.sin(2 * np.pi * x / 24))
test['time_cos'] = test['time'].apply(lambda x: np.cos(2 * np.pi * x / 24))

In [ ]:
# month 처리
## 겨울(0), 봄(1), 여름(2), 가을(3) 계절별로 나눔

conditions = [
    (test['fog_test.month'] == 12) | (test['fog_test.month'] <= 2), # 겨울
    (test['fog_test.month'] >= 3) & (test['fog_test.month'] <= 5), # 봄
    (test['fog_test.month'] >= 6) & (test['fog_test.month'] <= 8), # 여름
    (test['fog_test.month'] >= 9) & (test['fog_test.month'] <= 11) # 가을
    ]

values = [0, 1, 2, 3]

test['season'] = np.select(conditions, values)

test['season'].value_counts()

season
1    66240
2    66240
3    65520
0    64800
Name: count, dtype: int64

In [ ]:
# 칼럼 정리2
## 필요없는 칼럼 버리기

test = test.drop(columns = ['fog_test.month', 'fog_test.time', 'fog_test.minute', 'time', 'fog_test.ws10_deg'])
print(test.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 262800 entries, 0 to 262799
Data columns (total 15 columns):
 #   Column                 Non-Null Count   Dtype         
---  ------                 --------------   -----         
 0   fog_test.stn_id        262800 non-null  object        
 1   fog_test.ws10_ms       262800 non-null  float64       
 2   fog_test.ta            262800 non-null  float64       
 3   fog_test.re            262800 non-null  float64       
 4   fog_test.hm            262800 non-null  float64       
 5   fog_test.sun10         262800 non-null  float64       
 6   fog_test.ts            262800 non-null  float64       
 7   fog_test.class         0 non-null       float64       
 8   datetime               262800 non-null  datetime64[ns]
 9   date                   262800 non-null  object        
 10  fog_test.ws10_deg_cos  262800 non-null  float64       
 11  fog_test.ws10_deg_sin  262800 non-null  float64       
 12  time_sin               262800 non-null  floa

In [ ]:
#1. 대기-지면 온도차

test['Tdiff'] = test['fog_test.ta'] - test['fog_test.ts']
test['Tdiff']

0        -0.9
1        -0.9
2        -0.8
3        -1.0
4        -1.0
         ... 
262795    2.6
262796    2.7
262797    2.7
262798    2.9
262799    3.0
Name: Tdiff, Length: 262800, dtype: float64

In [ ]:
#2. 5시간 기온변화량

# 관측소 별로 묶어서 계산
test['ta5'] = test.groupby('fog_test.stn_id')['fog_test.ta'].diff(periods=30)
test['ta5'] # 첫 6x5=30 행은 NaN 값 발생

0         NaN
1         NaN
2         NaN
3         NaN
4         NaN
         ... 
262795   -1.8
262796   -1.6
262797   -1.6
262798   -1.8
262799   -1.8
Name: ta5, Length: 262800, dtype: float64

In [ ]:
# 젤 처음으로 나오는 유효값과 일치시킨다.

# 인덱스
first_valid_index = test['ta5'].first_valid_index()

# 값
first_valid_value = test.loc[first_valid_index, 'ta5']

# 채우기
test['ta5'].fillna(first_valid_value, inplace=True)

test['ta5'] # 해결완료

0        -3.4
1        -3.4
2        -3.4
3        -3.4
4        -3.4
         ... 
262795   -1.8
262796   -1.6
262797   -1.6
262798   -1.8
262799   -1.8
Name: ta5, Length: 262800, dtype: float64

In [ ]:
#3. 일사량

test['daily_rad'] = test.groupby(['fog_test.stn_id', 'date'])['fog_test.sun10'].transform('sum')

In [ ]:
#4. 이슬점온도

## Magnus 수식 쓸건데 log(상대습도/100) -> log 0 나오면 계산안됨
test['fog_test.hm'].min() # 상대습도 0인 데이터가 있음..

7.9

In [ ]:
## Magnus-Tetens formula

b = 17.62
c = 243.12

gamma = (b * test['fog_test.ta'] /(c + test['fog_test.ta'])) + np.log(test['fog_test.hm'] / 100.0 + 1e-10) # 아주 작은 수를 더해준다. (결과 큰 영향없음)
test['td'] = (c * gamma) / (b - gamma)
test['td']

0        -4.291327
1        -4.185183
2        -4.124864
3        -4.208489
4        -4.163491
            ...   
262795   -0.249011
262796   -0.133495
262797   -0.095276
262798   -0.211744
262799    0.014583
Name: td, Length: 262800, dtype: float64

In [ ]:
#5. 포차 (dew point depression)
## 포차 = 기온 - 이슬점 (상대습도 개념과 유사, 얼마나 포화되어있는가)

test['td_dep'] = test['fog_test.ta'] - test['td']
test['td_dep']

0         1.791327
1         1.685183
2         1.624864
3         1.608489
4         1.563491
            ...   
262795    4.749011
262796    4.733495
262797    4.595276
262798    4.511744
262799    4.285417
Name: td_dep, Length: 262800, dtype: float64

In [ ]:
#7. 전날 강수강도

## 일별 강수강도 = 24시간 강수유무(0,1)의 평균

# 각 관측소 + 일자 별로 일별 강수강도 계산하여 저장
daily_rain_power = test.groupby(['fog_test.stn_id', 'date'])['fog_test.re'].mean()

# 전날 강수강도를 'rain_pre_power' 열에 할당
test['re_pow_pre'] = test.apply(lambda row: daily_rain_power.get((row['fog_test.stn_id'], row['datetime'].date() - pd.Timedelta(days=1)), 0), axis=1)
## NaN 값이 발생할걸로 예상되는 첫날도 0으로 채워줌.
## test.apply 써서 좀 느릴 수 있음. (2분 소요)

In [ ]:
print(daily_rain_power.describe())
test['re_pow_pre']

count    1825.000000
mean        0.061838
std         0.162839
min         0.000000
25%         0.000000
50%         0.000000
75%         0.013889
max         1.000000
Name: fog_test.re, dtype: float64


0         0.000000
1         0.000000
2         0.000000
3         0.000000
4         0.000000
            ...   
262795    0.006944
262796    0.006944
262797    0.006944
262798    0.006944
262799    0.006944
Name: re_pow_pre, Length: 262800, dtype: float64

In [ ]:
#8. 당일 최고기온

# 관측소(stn_id)별로 각 날짜의 최고기온을 구하여 'ta_max' 칼럼에 추가
test['ta_max'] = test.groupby(['fog_test.stn_id', 'date'])['fog_test.ta'].transform('max')

In [ ]:
test['ta_max']

0         6.2
1         6.2
2         6.2
3         6.2
4         6.2
         ... 
262795    9.3
262796    9.3
262797    9.3
262798    9.3
262799    9.3
Name: ta_max, Length: 262800, dtype: float64

In [ ]:
#8. 당일 최저기온

# 관측소(stn_id)별로 각 날짜의 최저기온을 구하여 'ta_min' 칼럼에 추가
test['ta_min'] = test.groupby(['fog_test.stn_id', 'date'])['fog_test.ta'].transform('min')

In [ ]:
test['ta_min']

0        -8.0
1        -8.0
2        -8.0
3        -8.0
4        -8.0
         ... 
262795    4.3
262796    4.3
262797    4.3
262798    4.3
262799    4.3
Name: ta_min, Length: 262800, dtype: float64

In [ ]:
test['ta_range'] = test['ta_max'] - test['ta_min']
test['ta_range']

0         14.2
1         14.2
2         14.2
3         14.2
4         14.2
          ... 
262795     5.0
262796     5.0
262797     5.0
262798     5.0
262799     5.0
Name: ta_range, Length: 262800, dtype: float64

In [ ]:
## 잘 구성되어있는지 확인

test.describe()

,fog_test.ws10_ms,fog_test.ta,fog_test.re,fog_test.hm,fog_test.sun10,fog_test.ts,fog_test.class,datetime,fog_test.ws10_deg_cos,fog_test.ws10_deg_sin,...,season,Tdiff,ta5,daily_rad,td,td_dep,re_pow_pre,ta_max,ta_min,ta_range
count,262800.000000,262800.000000,262800.000000,262800.000000,262800.000000,262800.000000,0.0,262800,262800.000000,262800.000000,...,262800.000000,262800.00000,262800.000000,262800.000000,262800.000000,262800.000000,262800.000000,262800.000000,262800.000000,262800.000000
mean,2.296262,13.803448,0.061838,67.019639,0.101570,15.638048,NaN,2023-07-02 11:54:59.999998720,0.036343,0.001337,...,1.504110,-1.83460,-0.000685,14.626145,6.938660,6.864788,0.061568,18.817699,9.487781,9.329918
min,0.000000,-20.100000,0.000000,7.900000,0.000000,-4.900000,NaN,2023-01-01 00:00:00,-1.000000,-0.999990,...,0.000000,-33.10000,-16.300000,0.600000,-33.617204,0.013793,0.000000,-9.000000,-20.100000,1.000000
25%,1.000000,5.600000,0.000000,51.100000,0.000000,4.600000,NaN,2023-04-02 05:57:30,-0.685172,-0.688411,...,1.000000,-3.20000,-2.400000,9.190000,-2.863827,2.310199,0.000000,10.800000,0.600000,6.100000
50%,2.000000,15.100000,0.000000,69.300000,0.000000,15.900000,NaN,2023-07-02 11:55:00,0.059276,0.000000,...,2.000000,-0.80000,-0.600000,13.900000,8.019389,5.516915,0.000000,20.600000,10.200000,8.600000
75%,3.100000,22.600000,0.000000,86.100000,0.170000,24.500000,NaN,2023-10-01 17:52:30,0.767695,0.685851,...,2.000000,1.20000,2.000000,20.540000,18.152854,9.790059,0.013889,26.500000,19.100000,11.700000
max,19.500000,38.300000,1.000000,99.900000,1.730000,67.400000,NaN,2023-12-31 23:50:00,1.000000,1.000000,...,3.000000,36.90000,23.200000,30.815000,27.050023,36.701574,1.000000,38.300000,30.600000,26.600000
std,1.791743,10.523528,0.240861,21.791007,0.155777,12.650214,NaN,NaN,0.718324,0.694760,...,1.114347,5.02624,4.250535,7.322177,12.157307,5.760433,0.162694,9.670283,10.738362,4.541576


In [ ]:
# vis1, class 칼럼 맨 뒤로 이동 (왜?) - > 보기 편해서
columns = [col for col in test.columns if col != 'fog_test.class'] + ['fog_test.class']
test = test[columns]
test

,fog_test.stn_id,fog_test.ws10_ms,fog_test.ta,fog_test.re,fog_test.hm,fog_test.sun10,fog_test.ts,datetime,date,fog_test.ws10_deg_cos,...,Tdiff,ta5,daily_rad,td,td_dep,re_pow_pre,ta_max,ta_min,ta_range,fog_test.class
0,AI,0.6,-2.5,0.0,87.5,0.0,-1.6,2023-01-01 00:00:00,2023-01-01,-0.933326,...,-0.9,-3.4,9.22,-4.291327,1.791327,0.000000,6.2,-8.0,14.2,NaN
1,AI,1.2,-2.5,0.0,88.2,0.0,-1.6,2023-01-01 00:10:00,2023-01-01,0.211634,...,-0.9,-3.4,9.22,-4.185183,1.685183,0.000000,6.2,-8.0,14.2,NaN
2,AI,0.4,-2.5,0.0,88.6,0.0,-1.7,2023-01-01 00:20:00,2023-01-01,0.921061,...,-0.8,-3.4,9.22,-4.124864,1.624864,0.000000,6.2,-8.0,14.2,NaN
3,AI,0.7,-2.6,0.0,88.7,0.0,-1.6,2023-01-01 00:30:00,2023-01-01,-0.999873,...,-1.0,-3.4,9.22,-4.208489,1.608489,0.000000,6.2,-8.0,14.2,NaN
4,AI,0.2,-2.6,0.0,89.0,0.0,-1.6,2023-01-01 00:40:00,2023-01-01,0.493328,...,-1.0,-3.4,9.22,-4.163491,1.563491,0.000000,6.2,-8.0,14.2,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
262795,ED,6.1,4.5,0.0,71.3,0.0,1.9,2023-12-31 23:10:00,2023-12-31,0.881387,...,2.6,-1.8,6.04,-0.249011,4.749011,0.006944,9.3,4.3,5.0,NaN
262796,ED,5.8,4.6,0.0,71.4,0.0,1.9,2023-12-31 23:20:00,2023-12-31,-0.581346,...,2.7,-1.6,6.04,-0.133495,4.733495,0.006944,9.3,4.3,5.0,NaN
262797,ED,5.3,4.5,0.0,72.1,0.0,1.8,2023-12-31 23:30:00,2023-12-31,-0.582020,...,2.7,-1.6,6.04,-0.095276,4.595276,0.006944,9.3,4.3,5.0,NaN
262798,ED,3.7,4.3,0.0,72.5,0.0,1.4,2023-12-31 23:40:00,2023-12-31,0.013247,...,2.9,-1.8,6.04,-0.211744,4.511744,0.006944,9.3,4.3,5.0,NaN


In [ ]:
test.to_csv('submission.csv', index=False)

# submission

In [ ]:
submission = pd.read_csv('/content/drive/MyDrive/기상청빅데이터/과제 2/Model/loadmodel/submission.csv')

In [ ]:
# train과 행 이름 같기해야됨.. 이름 잘 지을
submission.columns = submission.columns.str.replace('test', 'train')

In [ ]:
columns_to_scale = ['fog_train.ws10_ms', 'fog_train.ta', 'fog_train.re', 'fog_train.hm', 'fog_train.sun10', 'fog_train.ts',
                    'fog_train.ws10_deg_cos', 'fog_train.ws10_deg_sin', 'time_sin', 'time_cos',
                    'season', 'Tdiff', 'ta5', 'td', 'td_dep',
                    're_pow_pre', 'ta_max', 'ta_min', 'ta_range', 'daily_rad']


submission[columns_to_scale] = sd_scaler.transform(submission[columns_to_scale])

In [ ]:
submission = submission.drop(columns = ['datetime', 'date'])
submission

,fog_train.stn_id,fog_train.ws10_ms,fog_train.ta,fog_train.re,fog_train.hm,fog_train.sun10,fog_train.ts,fog_train.ws10_deg_cos,fog_train.ws10_deg_sin,time_sin,...,Tdiff,ta5,daily_rad,td,td_dep,re_pow_pre,ta_max,ta_min,ta_range,fog_train.class
0,AI,-0.902045,-1.550966,-0.236137,0.982467,-0.64776,-1.381526,-1.341567,0.509588,-4.463317e-07,...,0.239672,-0.836033,-0.750807,-0.930884,-0.938602,-0.417904,-1.283775,-1.645117,1.213169,NaN
1,AI,-0.561175,-1.550966,-0.236137,1.015794,-0.64776,-1.381526,0.258872,1.395174,1.027996e-03,...,0.239672,-0.836033,-0.750807,-0.921857,-0.958589,-0.417904,-1.283775,-1.645117,1.213169,NaN
2,AI,-1.015669,-1.550966,-0.236137,1.034838,-0.64776,-1.389590,1.250517,0.553112,2.056437e-03,...,0.259611,-0.836033,-0.750807,-0.916727,-0.969947,-0.417904,-1.283775,-1.645117,1.213169,NaN
3,AI,-0.845233,-1.560701,-0.236137,1.039599,-0.64776,-1.381526,-1.434587,-0.027485,3.084877e-03,...,0.219732,-0.836033,-0.750807,-0.923839,-0.973030,-0.417904,-1.283775,-1.645117,1.213169,NaN
4,AI,-1.129292,-1.560701,-0.236137,1.053882,-0.64776,-1.381526,0.652627,1.241199,4.113316e-03,...,0.219732,-0.836033,-0.750807,-0.920012,-0.981503,-0.417904,-1.283775,-1.645117,1.213169,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
262795,ED,2.222602,-0.869471,-0.236137,0.211193,-0.64776,-1.099299,1.195060,-0.681219,-3.650313e-01,...,0.937557,-0.442752,-1.184993,-0.587115,-0.381680,-0.362954,-0.959352,-0.458556,-1.118668,NaN
262796,ED,2.052166,-0.859735,-0.236137,0.215954,-0.64776,-1.099299,-0.849566,1.160725,-3.640376e-01,...,0.957496,-0.393592,-1.184993,-0.577291,-0.384602,-0.362954,-0.959352,-0.458556,-1.118668,NaN
262797,ED,1.768108,-0.869471,-0.236137,0.249280,-0.64776,-1.107362,-0.850507,-1.169299,-3.630437e-01,...,0.957496,-0.393592,-1.184993,-0.574041,-0.410628,-0.362954,-0.959352,-0.458556,-1.118668,NaN
262798,ED,0.859119,-0.888942,-0.236137,0.268324,-0.64776,-1.139617,-0.018437,1.427490,-3.620496e-01,...,0.997375,-0.442752,-1.184993,-0.583945,-0.426357,-0.362954,-0.959352,-0.458556,-1.118668,NaN


In [ ]:
submission_A = submission[submission['fog_train.stn_id'].str.startswith('A')]
submission_B = submission[submission['fog_train.stn_id'].str.startswith('B')]
submission_C = submission[submission['fog_train.stn_id'].str.startswith('C')]
submission_D = submission[submission['fog_train.stn_id'].str.startswith('D')]
submission_E = submission[submission['fog_train.stn_id'].str.startswith('E')]

# 이제 쓸모가 없는 관측소 데이터 삭제
submission_A = submission_A.drop(columns = ['fog_train.stn_id'])
submission_B = submission_B.drop(columns = ['fog_train.stn_id'])
submission_C = submission_C.drop(columns = ['fog_train.stn_id'])
submission_D = submission_D.drop(columns = ['fog_train.stn_id'])
submission_E = submission_E.drop(columns = ['fog_train.stn_id'])

In [ ]:
from xgboost import XGBClassifier

In [ ]:
PCA_df_submission_A = submission_A.drop(columns = ['fog_train.re', 'fog_train.hm', 'td_dep', 'fog_train.class'])
PCA_df_submission_A = pca_model_A.transform(PCA_df_submission_A)
principalDf = pd.DataFrame(data = PCA_df_submission_A, columns = ['PC1', 'PC2', 'PC3', 'PC4', 'PC5'])
X_A = pd.concat([submission_A[['fog_train.re', 'fog_train.hm', 'td_dep']].reset_index(drop=True), principalDf.reset_index(drop=True)], axis = 1)
y_A = submission_A['fog_train.class']

In [ ]:
y_A = xgb_model_A.predict(X_A)

In [ ]:
PCA_df_submission_B = submission_B.drop(columns = ['fog_train.re', 'fog_train.hm', 'td_dep', 'fog_train.class'])
PCA_df_submission_B = pca_model_B.transform(PCA_df_submission_B)
principalDf = pd.DataFrame(data = PCA_df_submission_B, columns = ['PC1', 'PC2', 'PC3', 'PC4', 'PC5'])
X_B = pd.concat([submission_B[['fog_train.re', 'fog_train.hm', 'td_dep']].reset_index(drop=True), principalDf.reset_index(drop=True)], axis = 1)
y_B = submission_B['fog_train.class']

In [ ]:
y_B = xgb_model_B.predict(X_B)

In [ ]:
PCA_df_submission_C = submission_C.drop(columns = ['fog_train.ta', 'fog_train.hm', 'td_dep', 'fog_train.class'])
PCA_df_submission_C = pca_model_C.transform(PCA_df_submission_C)
principalDf = pd.DataFrame(data = PCA_df_submission_C, columns = ['PC1', 'PC2', 'PC3', 'PC4', 'PC5'])
X_C = pd.concat([submission_C[['fog_train.ta', 'fog_train.hm', 'td_dep']].reset_index(drop=True), principalDf.reset_index(drop=True)], axis = 1)
y_C = submission_C['fog_train.class']

In [ ]:
y_C = xgb_model_C.predict(X_C)

In [ ]:
PCA_df_submission_D = submission_D.drop(columns = ['fog_train.sun10', 'fog_train.hm', 'td_dep', 'fog_train.class'])
PCA_df_submission_D = pca_model_D.transform(PCA_df_submission_D)
principalDf = pd.DataFrame(data = PCA_df_submission_D, columns = ['PC1', 'PC2', 'PC3', 'PC4', 'PC5'])
X_D = pd.concat([submission_D[['fog_train.sun10', 'fog_train.hm', 'td_dep']].reset_index(drop=True), principalDf.reset_index(drop=True)], axis = 1)
y_D = submission_D['fog_train.class']

In [ ]:
y_D = xgb_model_D.predict(X_D)

In [ ]:
PCA_df_submission_E = submission_E.drop(columns = ['fog_train.re', 'fog_train.hm', 'td_dep', 'fog_train.class'])
PCA_df_submission_E = pca_model_E.transform(PCA_df_submission_E)
principalDf = pd.DataFrame(data = PCA_df_submission_E, columns = ['PC1', 'PC2', 'PC3', 'PC4', 'PC5'])
X_E = pd.concat([submission_E[['fog_train.re', 'fog_train.hm', 'td_dep']].reset_index(drop=True), principalDf.reset_index(drop=True)], axis = 1)
y_E = submission_E['fog_train.class']

In [ ]:
y_E = xgb_model_E.predict(X_E)

In [ ]:
y = np.concatenate((y_A, y_B, y_C, y_D, y_E))
len(y)

262800

In [ ]:
y = y+1

In [ ]:
print(y)

[4 4 4 ... 4 4 4]


In [ ]:
unique, counts = np.unique(y, return_counts=True)

# Create a dictionary of unique elements and their counts
count_dict = dict(zip(unique, counts))

# Display the count dictionary
print(count_dict)

{1: 819, 2: 2994, 3: 3853, 4: 255134}


# final

In [ ]:
final = pd.read_csv('/content/drive/MyDrive/기상청빅데이터/과제 2/Model/loadmodel/fog_test.csv')

In [ ]:
len(final)

262800

In [ ]:
final.isnull().sum()

fog_test.year             0
fog_test.month            0
fog_test.day              0
fog_test.time             0
fog_test.minute           0
fog_test.stn_id           0
fog_test.ws10_deg         0
fog_test.ws10_ms          0
fog_test.ta               0
fog_test.re               0
fog_test.hm               0
fog_test.sun10            0
fog_test.ts               0
fog_test.class       260106
dtype: int64

In [ ]:
# Updating the 'fog_test.class' column with 'y' values while preserving -99 values
final['fog_test.class'] = final['fog_test.class'].where(final['fog_test.class'] == -99, y)
final['fog_test.class'] = final['fog_test.class'].astype(int)

In [ ]:
final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 262800 entries, 0 to 262799
Data columns (total 14 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   fog_test.year      262800 non-null  object 
 1   fog_test.month     262800 non-null  int64  
 2   fog_test.day       262800 non-null  int64  
 3   fog_test.time      262800 non-null  int64  
 4   fog_test.minute    262800 non-null  int64  
 5   fog_test.stn_id    262800 non-null  object 
 6   fog_test.ws10_deg  262800 non-null  float64
 7   fog_test.ws10_ms   262800 non-null  float64
 8   fog_test.ta        262800 non-null  float64
 9   fog_test.re        262800 non-null  float64
 10  fog_test.hm        262800 non-null  float64
 11  fog_test.sun10     262800 non-null  float64
 12  fog_test.ts        262800 non-null  float64
 13  fog_test.class     262800 non-null  int64  
dtypes: float64(7), int64(5), object(2)
memory usage: 28.1+ MB


In [ ]:
final.isnull().sum()

fog_test.year        0
fog_test.month       0
fog_test.day         0
fog_test.time        0
fog_test.minute      0
fog_test.stn_id      0
fog_test.ws10_deg    0
fog_test.ws10_ms     0
fog_test.ta          0
fog_test.re          0
fog_test.hm          0
fog_test.sun10       0
fog_test.ts          0
fog_test.class       0
dtype: int64

In [ ]:
final.to_csv("240041.csv", index = False)

In [ ]:
final

,fog_test.year,fog_test.month,fog_test.day,fog_test.time,fog_test.minute,fog_test.stn_id,fog_test.ws10_deg,fog_test.ws10_ms,fog_test.ta,fog_test.re,fog_test.hm,fog_test.sun10,fog_test.ts,fog_test.class
0,L,1,1,0,0,AI,329.5,0.6,-2.5,0.0,87.5,0.0,-1.6,4
1,L,1,1,0,10,AI,321.8,1.2,-2.5,0.0,88.2,0.0,-1.6,4
2,L,1,1,0,20,AI,0.4,0.4,-2.5,0.0,88.6,0.0,-1.7,4
3,L,1,1,0,30,AI,323.6,0.7,-2.6,0.0,88.7,0.0,-1.6,4
4,L,1,1,0,40,AI,208.4,0.2,-2.6,0.0,89.0,0.0,-1.6,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
262795,L,12,31,23,10,ED,338.8,6.1,4.5,0.0,71.3,0.0,1.9,4
262796,L,12,31,23,20,ED,335.2,5.8,4.6,0.0,71.4,0.0,1.9,4
262797,L,12,31,23,30,ED,337.1,5.3,4.5,0.0,72.1,0.0,1.8,4
262798,L,12,31,23,40,ED,322.0,3.7,4.3,0.0,72.5,0.0,1.4,4
